In [1]:
import os
import random
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tempfile import TemporaryDirectory

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, random_split

import torchvision
from torchvision import datasets, models, transforms

import wandb

cudnn.benchmark = True
plt.ion()

!wandb login d77624ba279c6354e2d27130c47fa3faf424ea9d


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [5]:
# Customed Dataset class
class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.classes = self._find_classes()
        self.image_paths, self.labels = self._load_data()

    def _find_classes(self):
        classes = sorted([d for d in os.listdir(self.data_dir) if os.path.isdir(os.path.join(self.data_dir, d))])
        return classes

    def _load_data(self):
        image_paths = []
        labels = []
        for label in self.classes:
            class_dir = os.path.join(self.data_dir, label)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                image_paths.append(img_path)
                labels.append(int(label))
        return image_paths, labels

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        label = int(label)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

In [7]:
# Parameters

# Change batchsize to fit hardware
batch_size = 128

# Training parameters
num_classes = 2139  
learning_rate = 0.001
num_epochs = 50

# Scheduler
step_size = 7
gamma = 0.1

In [8]:
# Location of data
train_dir = '/kaggle/input/wb-recognition-dataset2/wb_recognition_dataset/train'
val_dir = '/kaggle/input/wb-recognition-dataset2/wb_recognition_dataset/val'

In [9]:
# Data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),  
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [10]:
# Datasets from each folder
image_datasets = {
    'train': CustomDataset(train_dir, data_transforms['train']),
    'val': CustomDataset(val_dir, data_transforms['val']),
}

# Dataloader iterators
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True, num_workers=4, pin_memory = True),
    'val': DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=False, num_workers=4, pin_memory = True),
}

# Size of datasets
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [13]:
# Number of images and labels
print('Number of images in train: ', dataset_sizes['train'])
print('Number of labels in train: ',len(image_datasets['train'].classes))
print('Number of images in val: ', dataset_sizes['val'])
print('Number of labels in val: ', len(image_datasets['val'].classes))

Number of images in train:  59009
Number of labels in train:  2130
Number of images in val:  1392
Number of labels in val:  595


In [14]:
pip install efficientnet-pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=cdbd7584c705c7701481960f9c1b919e87bbe59434afabfae4baaa15981e7e8c
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet-pytorch
Note: you may need to restart the kernel to use updated packages.


In [16]:
from efficientnet_pytorch import EfficientNet
# Load pretrained model
model = EfficientNet.from_pretrained('efficientnet-b0')

# Replace fully connected layer
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, num_classes)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 48.4MB/s]


Loaded pretrained weights for efficientnet-b0


In [17]:
#adam
# Training loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Decay the learning rate by 10% every 7 epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size= step_size, gamma=gamma)

In [18]:
# Move to gpu 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

cuda:0


In [19]:
# Check model
print(model)

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [20]:
def train_model(model, criterion, optimizer, dataloaders, model_last_pth, model_best_pth, num_epochs):
    """Train a PyTorch Model

    Params
    --------
        model (PyTorch model): cnn to train
        criterion (PyTorch loss): objective to minimize
        optimizer (PyTorch optimizier): optimizer to compute gradients of model parameters
        dataloaders (PyTorch dataloader): dataloaders to iterate through
        model_last_pth, model_best_pth (str ending in '.pt'): file path to save the model state dict
        num_epochs (int): maximum number of training epochs

    Returns
    --------
        model (PyTorch model): trained cnn with best weights
    """
    
    # Min validation loss
    valid_loss_min = np.Inf
    
    # Main loop
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1} / {num_epochs}')
        print('-' * 10)
        
        # Go through training and validation phase each epoch
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            # Keep track of loss and corrects each epoch
            running_loss = 0.0
            running_corrects = 0

            # Training loop
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                
                # Clear gradients
                optimizer.zero_grad()
                
                # Predicted outputs and loss of gradients
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backpropagation and update parameters
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Update loss and number of correct predictions
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            # Step the scheduler if in training phase
            if phase == 'train':
                scheduler.step()
            
            # Calculate loss and accuracy of each epoch
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            
            if phase == 'val':
                if epoch_loss < valid_loss_min:
                    valid_loss_min = epoch_loss
                    torch.save(model.state_dict(), model_best_pth)
            
            print(f'{phase}\t Loss: {epoch_loss:.4f}\t Accuracy: {epoch_acc:.4f}')
            
            wandb.log({f'{phase}_loss': epoch_loss, f'{phase}_acc': epoch_acc})
        
        # Save model every epoch
        torch.save(model.state_dict(), model_last_pth)

    return model

In [21]:
# Path to save model (last and best)
model_last_pth = '/kaggle/working/efficientNetb0-imagenet-01-last.pt'
model_best_pth = '/kaggle/working/efficientNetb0-imagenet-01-best.pt'

# Saved model 
saved_model_path = '/kaggle/working/efficientNetb0-imagenet-01-best.pt'

In [24]:
wandb.init(project='ImageProcessing-project', sync_tensorboard=True)

# Load saved model
try:
    model.load_state_dict(torch.load(saved_model_path))
    print('Loaded saved model successfully')
except FileNotFoundError:
    print('File not found')
except Exception as e:
    print(f'An error occurred: {e}')

print()

model.to(device)
    
model = train_model(model, criterion, optimizer, dataloaders, model_last_pth, model_best_pth, num_epochs)

wandb.finish()

train_acc,▁▇▇▇██████████████████
train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▆▆▇▇▇▁▇███████████████
val_loss,▂▂▂▁▂█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,0.99778
train_loss,0.01021
val_acc,0.95187
val_loss,0.24324


Loaded saved model successfully

Epoch 1 / 50
----------
train	 Loss: 0.0134	 Accuracy: 0.9971
val	 Loss: 0.2342	 Accuracy: 0.9497
Epoch 2 / 50
----------
train	 Loss: 0.0133	 Accuracy: 0.9972
val	 Loss: 0.2345	 Accuracy: 0.9511
Epoch 3 / 50
----------
train	 Loss: 0.0129	 Accuracy: 0.9974
val	 Loss: 0.2342	 Accuracy: 0.9511
Epoch 4 / 50
----------
train	 Loss: 0.0128	 Accuracy: 0.9973
val	 Loss: 0.2319	 Accuracy: 0.9511
Epoch 5 / 50
----------
train	 Loss: 0.0133	 Accuracy: 0.9971
val	 Loss: 0.2339	 Accuracy: 0.9526
Epoch 6 / 50
----------
train	 Loss: 0.0129	 Accuracy: 0.9972
val	 Loss: 0.2332	 Accuracy: 0.9533
Epoch 7 / 50
----------
train	 Loss: 0.0125	 Accuracy: 0.9974
val	 Loss: 0.2339	 Accuracy: 0.9519
Epoch 8 / 50
----------
train	 Loss: 0.0126	 Accuracy: 0.9976
val	 Loss: 0.2341	 Accuracy: 0.9526
Epoch 9 / 50
----------
train	 Loss: 0.0126	 Accuracy: 0.9974
val	 Loss: 0.2345	 Accuracy: 0.9526
Epoch 10 / 50
----------
train	 Loss: 0.0127	 Accuracy: 0.9973
val	 Loss: 0.2349	 Acc

train_acc,▁▂▆▄▃▆█▆▆▅▆▄▆▆▅▆▄▄▇▆▅█▇▅▃▆▄▇▄▆▇▆▅▇▇▅▆▅▆▅
train_loss,██▅▄▅▃▃▃▂▅▄▃▄▂▃▃▄▅▂▃▄▁▄▄▃▃▃▃▅▃▂▂▅▃▃▃▃▄▄▃
val_acc,▁▄▄▄█▅▇▇▇▅▇▇▅▅▄▅▅▅▂▅▇▅▇▅▅▅▄▄▇▅▅▇▅▅▅▇▇▄▅▇
val_loss,▆▆▆▁▄▅▅▆▅▆▆▆▃▅▃▅▄▆▄▆▅▄▆▆▃▇▅▄▄█▅▅▄▆▇▅▄▄▅▅
train_acc,0.99736
train_loss,0.01254
val_acc,0.95259
val_loss,0.23383


In [23]:
#sgd
# Training loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.9)

# Decay the learning rate by 10% every 7 epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size= step_size, gamma=gamma)